### FCN Black-box Classifier

In [1]:
import tensorflow as tf
from tensorflow import keras
import time
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior() # disable TF2 behaviour as alibi code still relies on TF1 constructs
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, GlobalAveragePooling1D, BatchNormalization, Conv2D
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.backend import function

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import sklearn
from scipy.optimize import minimize
from scipy.spatial.distance import cdist, pdist
from scipy import stats
from sklearn.neighbors import DistanceMetric
from tslearn.datasets import UCR_UEA_datasets
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from counterfactual_utils import label_encoder, ucr_data_loader
print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False
%matplotlib inline


TF version:  2.5.0
Eager execution enabled:  False
TF version:  2.5.0
Eager execution enabled:  False


#### Classifier Architecture

This architecture was originally proposed by Wang et al and the implementation closely follows the code provided by Fawaz et al in a fantastic review paper on DNNs for Time series classification.

In [2]:
class Classifier_FCN:

    def __init__(self, output_directory, input_shape, nb_classes, dataset_name, model_suffix='',verbose=False,build=True):
        self.output_directory = output_directory
        self.dataset_name = str(dataset_name)   # ✅ 保存为实例属性，避免依赖外部同名变量
        self.model_suffix = model_suffix        # ✅ 保存后缀
        self.verbose = verbose                  # ✅ 先保存，和原逻辑一致

        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if(verbose==True):
                self.model.summary()
            # ✅ 初始权重文件名加后缀；仍保存在当前目录
            self.model.save_weights(self.dataset_name + f'_model_init{self.model_suffix}.hdf5')
        return

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        conv1 = keras.layers.Conv1D(filters=128, kernel_size=8, padding='same')(input_layer)
        conv1 = keras.layers.BatchNormalization()(conv1)
        conv1 = keras.layers.Activation(activation='relu')(conv1)

        conv2 = keras.layers.Conv1D(filters=256, kernel_size=5, padding='same')(conv1)
        conv2 = keras.layers.BatchNormalization()(conv2)
        conv2 = keras.layers.Activation('relu')(conv2)

        conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
        conv3 = keras.layers.BatchNormalization()(conv3)
        conv3 = keras.layers.Activation('relu')(conv3)

        gap_layer = keras.layers.GlobalAveragePooling1D()(conv3)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(),
            metrics=['accuracy'])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50,
            min_lr=0.0001)

        # ✅ best 文件名加后缀；仍保存在当前目录
        file_path = self.dataset_name + f'_best_model{self.model_suffix}.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
            save_best_only=True)

        self.callbacks = [reduce_lr,model_checkpoint]

        return model

    def fit(self, x_train, y_train):

        batch_size = 16
        nb_epochs = 2000

        mini_batch_size = int(min(x_train.shape[0]/10, batch_size))


        hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
            verbose=self.verbose, callbacks=self.callbacks)

        # ✅ last 文件名加后缀；仍保存在当前目录
        self.model.save(self.dataset_name + f'_last_model{self.model_suffix}.hdf5')

        # ✅ 加载 best 的文件名加后缀；仍保存在当前目录
        model = keras.models.load_model(self.dataset_name + f'_best_model{self.model_suffix}.hdf5')
        # （这里原项目也是加载后未使用，我们保持一致，不做额外修改）


    def predict(self, x_test):
        # ✅ 预测时加载的 best 文件名加后缀；仍保存在当前目录
        model_path = self.dataset_name + f'_best_model{self.model_suffix}.hdf5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test)
        y_pred = np.argmax(y_pred, axis=1)
        return y_pred


In [ ]:
# class Classifier_FCN:
#
#     def __init__(self, output_directory, input_shape, nb_classes, dataset_name, verbose=False,build=True):
#         self.output_directory = output_directory
#
#
#         if build == True:
#             self.model = self.build_model(input_shape, nb_classes)
#             if(verbose==True):
#                 self.model.summary()
#             self.verbose = verbose
#             self.model.save_weights(str(dataset_name) +'_model_init.hdf5')
#         return
#
#     def build_model(self, input_shape, nb_classes):
#         input_layer = keras.layers.Input(input_shape)
#
#         conv1 = keras.layers.Conv1D(filters=128, kernel_size=8, padding='same')(input_layer)
#         conv1 = keras.layers.BatchNormalization()(conv1)
#         conv1 = keras.layers.Activation(activation='relu')(conv1)
#
#         conv2 = keras.layers.Conv1D(filters=256, kernel_size=5, padding='same')(conv1)
#         conv2 = keras.layers.BatchNormalization()(conv2)
#         conv2 = keras.layers.Activation('relu')(conv2)
#
#         conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
#         conv3 = keras.layers.BatchNormalization()(conv3)
#         conv3 = keras.layers.Activation('relu')(conv3)
#
#         gap_layer = keras.layers.GlobalAveragePooling1D()(conv3)
#
#         output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)
#
#         model = keras.models.Model(inputs=input_layer, outputs=output_layer)
#
#         model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(),
#             metrics=['accuracy'])
#
#         reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50,
#             min_lr=0.0001)
#
#         file_path = str(dataset_name) +'_best_model.hdf5'
#
#         model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
#             save_best_only=True)
#
#         self.callbacks = [reduce_lr,model_checkpoint]
#
#         return model
#
#     def fit(self, x_train, y_train):
#
#         batch_size = 16
#         nb_epochs = 2000
#
#         mini_batch_size = int(min(x_train.shape[0]/10, batch_size))
#
#
#         hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
#             verbose=self.verbose, callbacks=self.callbacks)
#
#         self.model.save(str(dataset_name) +'_last_model.hdf5')
#
#         model = keras.models.load_model(str(dataset_name) +'_best_model.hdf5')
#
#
#     def predict(self, x_test):
#         model_path = str(dataset_name) + '_best_model.hdf5'
#         model = keras.models.load_model(model_path)
#         y_pred = model.predict(x_test)
#         y_pred = np.argmax(y_pred, axis=1)
#         return y_pred

#### Training and saving weights 

In [15]:
# for dataset in ['ecg200', 'gunpoint', 'coffee', 'chinatown', 'CBF']:
#
#    X_train, y_train, X_test, y_test = ucr_data_loader(str(dataset))
#    y_train, y_test = label_encoder(y_train, y_test)
#
#    input_shape = X_train.shape[1:]
#    nb_classes = len(np.unique(np.concatenate([y_train,y_test])))
#    one_hot = to_categorical(y_train)
#    dataset_name = str(dataset)
#
#
#    fcn = Classifier_FCN(output_directory=os.getcwd(), input_shape=input_shape, nb_classes=nb_classes, dataset_name=dataset_name)
#    fcn.build_model(input_shape=input_shape, nb_classes=nb_classes)
#    fcn.fit(X_train, to_categorical(y_train))
#    fcn.predict(X_test)

In [3]:
dataset = 'Lightning2'
#custom_suffix = '_myrun'   # 你想要的后缀
X_train, y_train, X_test, y_test = ucr_data_loader(dataset)
y_train, y_test = label_encoder(y_train, y_test)

# 注意：若 X_* 是 (N,T)，Conv1D 需要 (N,T,1)
if X_train.ndim == 2:
    X_train = X_train[..., np.newaxis]
    X_test  = X_test[...,  np.newaxis]

input_shape = X_train.shape[1:]
nb_classes  = len(np.unique(np.concatenate([y_train, y_test])))

fcn = Classifier_FCN(output_directory=os.getcwd(),
                     input_shape=input_shape,
                     nb_classes=nb_classes,
                     dataset_name=dataset,
                     model_suffix=custom_suffix,  # 只多这一项
                     verbose=True, build=True)

fcn.fit(X_train, to_categorical(y_train))
y_pred = fcn.predict(X_test)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 637, 1)]          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 637, 128)          1152      
_________________________________________________________________
batch_normalization (BatchNo (None, 637, 128)          512       
_________________________________________________________________
activation (Activation)      (None, 637, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 637, 256)          164096    
_________________________________________________________________
batch_normalization_1 (Batch (None, 637, 256)          1024      
_________________________________________________________________
activation_1 (Activation)    (None, 637, 256)          0     

C:\Users\myz\.conda\envs\ibcfe-tsc\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [4]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)


X_train shape: (60, 637, 1)
y_train shape: (60,)


In [5]:
print("First 5 samples of X_train:", X_train[:5])
print("First 5 labels of y_train:", y_train[:5])


First 5 samples of X_train: [[[-0.30335461]
  [-0.35632932]
  [-0.4501614 ]
  ...
  [-0.30939695]
  [-0.34403114]
  [-0.32867462]]

 [[-1.3960051 ]
  [-1.3960051 ]
  [-1.3960051 ]
  ...
  [-1.3960051 ]
  [-1.3960051 ]
  [-1.3960051 ]]

 [[-0.74631436]
  [-0.74631436]
  [-0.74631436]
  ...
  [-0.74631436]
  [-0.74631436]
  [-0.74631436]]

 [[-0.49483586]
  [-0.49483586]
  [-0.49483586]
  ...
  [-0.49483586]
  [-0.49483586]
  [-0.49483586]]

 [[-0.49881132]
  [-0.49881132]
  [-0.49881132]
  ...
  [-0.49881132]
  [-0.49881132]
  [-0.49881132]]]
First 5 labels of y_train: [1 0 0 0 1]


In [9]:
# X_train, y_train, X_test, y_test = ucr_data_loader(str(dataset))
# print("X_train shape:", X_train.shape)


X_train shape: (100, 96, 1)


In [21]:
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

DATASET = 'coffee'  # 改成你的
# 注意形状 (N,T,1)
X_train, y_train, X_test, y_test = ucr_data_loader(DATASET)
y_train, y_test = label_encoder(y_train, y_test)
if X_test.ndim == 2: X_test = X_test[..., np.newaxis]

# 1) 原作者模型（仓库自带best）
orig = load_model(f'./{DATASET}_best_model.hdf5')

# 2) 你训练的模型（带你的后缀；若存在于当前目录，改相对路径即可）
mine = load_model(f'./{DATASET}_best_model_myrun.hdf5')

y_pred_o = np.argmax(orig.predict(X_test), 1)
y_pred_m = np.argmax(mine.predict(X_test), 1)

acc_o = accuracy_score(y_test, y_pred_o)
acc_m = accuracy_score(y_test, y_pred_m)
agree_rate = (y_pred_o == y_pred_m).mean()
diff_idx = np.where(y_pred_o != y_pred_m)[0]

print("Original acc:", acc_o)
print("Mine acc    :", acc_m)
print("Prediction agreement:", agree_rate)
print("First few diff indices:", diff_idx[:10])

# 可选：更细指标
print(classification_report(y_test, y_pred_o))
print(classification_report(y_test, y_pred_m))
print(confusion_matrix(y_test, y_pred_o))
print(confusion_matrix(y_test, y_pred_m))


C:\Users\myz\.conda\envs\ibcfe-tsc\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Original acc: 1.0
Mine acc    : 1.0
Prediction agreement: 1.0
First few diff indices: []
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00        13

    accuracy                           1.00        28
   macro avg       1.00      1.00      1.00        28
weighted avg       1.00      1.00      1.00        28

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00        13

    accuracy                           1.00        28
   macro avg       1.00      1.00      1.00        28
weighted avg       1.00      1.00      1.00        28

[[15  0]
 [ 0 13]]
[[15  0]
 [ 0 13]]
